# Udacity Capstone Project: Sparify

## Part 1: Project Overview and Problem Statement

Many online services make use of subscriptions where users pay monthly for a product that they are using. When the platform is online, it can be profitable to implement a "freemium" model where the service is free to the population (usually with ads) and the subscription allows for the removal of ads.

In such situations, being able to predict when a user will unsubscribe so that they can be targeted with promotions to try and convince them to stay can reduce loss of potential revenue over time. In this dataset, the actions for a given user have been recorded and will be used towards this end.

However, online platforms tend to generate high volumes of data that can be difficult to keep in memory. Big data solutions such as Spark can help to distribute the workload among several computers, allowing machine learning to be done on larger datasets.

Thus, we will use Spark in this project to help with our aim of creating a ML model to predict churn on a user level.

## Part 2: Analysis

Before doing any machine learning, it is important to load the dataset, perform some cleaning, and do some basic investigation.

Depending on correlations, we can decide which features we would like to use in our model (feature selection) and we can also decide which features can be combined or infered to create new features that might provide useful information (feature engineering).


### Data Importation and Workspace Setup

Below, packages required for this problem will are imported as well as other prepwork for the project.

In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Cleaning and analysis will be done on this smaller dataset before doing anything with the larger dataset.

In [1]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col

In [2]:
# create a Spark session
spark = SparkSession.builder.master("local").appName("Sparkify").getOrCreate()

In [3]:
df = spark.read.json("mini_sparkify_event_data.json")
df.persist();

In [4]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [5]:
df.head(1)

[Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30')]

We can already see that there are 12 columns in our data set.

### Data Cleaning

Now that the data has been imported, the first and most obvious thing to check for is missing and duplicated values. 

Since I am relatively comfortable with SQL, I will try and use that a little. In order to do that I need to create a temp view within the spark object that created the dataframe using the `createOrReplaceTempView` method.

In [6]:
# Create Temp View
df.createOrReplaceTempView("df")

Now that this is complete, we can look at the null values for each column.

In [7]:
null_dict = {}

for i in df.columns:
    
    nulls = spark.sql("SELECT SUM(CASE WHEN {} IS NULL THEN 1 ELSE 0 END) AS null_{} \
                    FROM df \
                    where {} is null".format(i, i, i)).collect()[0]["null_{}".format(i)]
    
    print(i, nulls)
    null_dict[i] = nulls

artist 58392
auth None
firstName 8346
gender 8346
itemInSession None
lastName 8346
length 58392
level None
location 8346
method None
page None
registration 8346
sessionId None
song 58392
status None
ts None
userAgent 8346
userId None


We can see that there are 58k rows where the artist and song names are null. This could be because in this instance, the page that was visited was not intended to play a song. 

We can also see that there are several columns which are never null;

* auth
* itemInSession
* level
* method
* page
* sessionId
* status
* ts
* userId

This tells us that we have users who commit actions during a session. So, the unique key should be something like (userId, ts). We can quickly verify that:

In [8]:
# Test for unique key
spark.sql("SELECT userId, ts, count(*) AS n_rows \
        FROM df \
        GROUP BY 1, 2\
        HAVING count(*) > 1 \
        ORDER BY 3 DESC \
        LIMIT 10").collect()

[Row(userId='98', ts=1538838475000, n_rows=3),
 Row(userId='95', ts=1539665400000, n_rows=3),
 Row(userId='100022', ts=1541463867000, n_rows=3),
 Row(userId='', ts=1538488477000, n_rows=2),
 Row(userId='118', ts=1540521007000, n_rows=2),
 Row(userId='152', ts=1543206099000, n_rows=2),
 Row(userId='30', ts=1543436963000, n_rows=2),
 Row(userId='36', ts=1539070836000, n_rows=2),
 Row(userId='9', ts=1539714540000, n_rows=2),
 Row(userId='132', ts=1542791668000, n_rows=2)]

In [9]:
# Find number of userid-timestamp combinations that have duplicated rows
spark.sql("WITH T AS (SELECT userId, ts, count(*) AS n_rows \
        FROM df \
        GROUP BY 1, 2\
        HAVING COUNT(*) > 1 \
        ORDER BY 3 DESC \
        ) \
        SELECT COUNT(*) FROM T").collect()

[Row(count(1)=783)]

We can see here that my assertion was wrong and that we have almost 800 userId-ts combinations that have at least 2 entries. We can view an example as well using the query below...

In [10]:
# Get an example with duplicated rows
spark.sql("SELECT * \
        FROM df \
        WHERE userId = 98\
        AND ts = 1538838475000").collect()

[Row(artist='Mase', auth='Logged In', firstName='Sawyer', gender='M', itemInSession=70, lastName='Larson', length=255.16363, level='free', location='Houston-The Woodlands-Sugar Land, TX', method='PUT', page='NextSong', registration=1538069638000, sessionId=493, song="Lookin' At Me [feat. Puff Daddy] (Album Version)", status=200, ts=1538838475000, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='98'),
 Row(artist=None, auth='Logged In', firstName='Sawyer', gender='M', itemInSession=71, lastName='Larson', length=None, level='free', location='Houston-The Woodlands-Sugar Land, TX', method='GET', page='Home', registration=1538069638000, sessionId=493, song=None, status=200, ts=1538838475000, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='98'),
 Row(artist=None, auth='Logged In', firstName='Sawyer', gen

In this example we can see that the user went to the next song before visiting the home page twice all with the same timestamp. It's a little difficult to know what to do in this situation.

If we were to be joining tables then not knowing the primary key would result in the duplication of values. We won't be doing any joins with this data and so that is not an issue.

However, it might introduct noise for our ML model if we have duplicate rows. Hence, it would be worth dropping page visits that happen multiple times for the same userId and ts. This is because visiting the same page within the same timestamp, while it may not strictly be duplicated, can probably be viewed as such as it provides no more interesting information about a user. 

In [11]:
# Find the unique key
spark.sql("WITH T AS (SELECT userId, ts, itemInSession, page, count(*) AS n_rows \
        FROM df \
        GROUP BY 1, 2, 3, 4\
        HAVING count(*) > 1 \
        ORDER BY 5 DESC \
        ) \
        SELECT COUNT(*) AS n_duplicated FROM T").collect()

[Row(n_duplicated=0)]

Finally we have found the unique key. It is userId, ts, itemInSession, page. This is because the for each user they might visit multiple pages within a certain timestamp. However, for each page that they load the itemInSession is iterated.

However, there is once instance where the itemInSession was duplicated, meaning that the unique key is only achieved once the page is also taken into consideration.

This is a little unexpected because it should not be technically possible to visit multiple pages within the same timestamp. So what we'll do is we'll rank the rows and keep only the timestamps with the lowest itemInSession count per user. (For the one event that was duplicated, we'll just pick a row at random.)

In [12]:
df_unique = spark.sql("WITH T AS (SELECT *, ROW_NUMBER() OVER (PARTITION BY userId, ts ORDER BY itemInSession) as rank\
                    FROM df ) \
                    SELECT * FROM T \
                    WHERE rank = 1")

# Create Temp View
df_unique.createOrReplaceTempView("df_unique")

In [13]:
# Verify clean successul
spark.sql("WITH T AS (SELECT userId, ts, itemInSession, count(*) AS n_rows \
        FROM df_unique \
        GROUP BY 1, 2, 3\
        HAVING count(*) > 1 \
        ORDER BY 4 DESC \
        ) \
        SELECT COUNT(*) AS n_duplicated FROM T").collect()

[Row(n_duplicated=0)]

Very good! We've gotten rid of events that happen on the same timestamp, which means that we've taken care of duplicated values.

In [14]:
df_unique.head(1)

[Row(artist=None, auth='Logged Out', firstName=None, gender=None, itemInSession=103, lastName=None, length=None, level='paid', location=None, method='GET', page='Home', registration=None, sessionId=141, song=None, status=200, ts=1538391913000, userAgent=None, userId='', rank=1)]

As investigated above, there are times when we get missing values. The only issue is that this can happen because the page is not playing an artist. Thus, we need to find a way to distinguish between instances where a missing value is acceptable and other times when it is not.

To remind ourselves of missing values, here are the instances where this was true below...

* artist: 58392
* firstName: 8346
* gender: 8346
* lastName: 8346
* length: 58392
* location: 8346
* registration: 8346
* song: 58392
* userAgent: 8346

We can see that the number of null `registration` are 8,346. This means that for the columns `firstName`, `gender`, `lastName`,
`location` and `userAgent` Likely do not contain any null values that cannot be explained by missing a registration.

The remaining two columns `song` and `artist` will be null then the page does not play music.

To assess whether it is necessary to take any action on nulls, the following tests should be carried out:

1. All registrations have non-null `firstName`, `gender`, `lastName`, `location` and `userAgent`
2. All non-registrations have null columns in these cases
3. All non-song-playing pages will have null `song` and `artist` and 
4. vice versa (as above)

Once the data cleaning has been completed, the steps will be assembled into a single function that can be used.

### Exploratory Data Analysis

Before doing any definitions, some questions about the dataset will be defined and answered. This will give us a better feel for the dataset.

Questions will be answered using relevant statistics .


### Define Churn

Now that the preliminary analysis is complete, I will create a column `Churn` to use as the label for your model. 


>Tip:  I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

# Feature Engineering

In this section, the features that were most promising will be assembled into a vector to use for training the model. 


- TODO: Write a script to extract the necessary features from the smaller subset of data
- TODO: Ensure that your script is scalable, using the best practices discussed in Lesson 3
- TODO: Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.